In [1]:
# import dependencies
import pandas as pd
import datetime as dt
import requests
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [2]:
# load in data 
weather = pd.read_csv("monthly_weather.csv")

In [3]:
alltrips = pd.read_csv("Shared_Micromobility_Vehicle_trips.csv")

C:\Users\Ashley\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
weather.head()

,Month,maxtemp,atemp,mintemp,maxdew,adew,mindew,maxhum,ahum,minhum,maxwind,awind,minwind,maxp,ap,minp,prec
0,1/1/2019,46,41.5,36,39,36.8,35,97,83.5,76,21,11.2,0,29.7,29.6,29.5,0
1,1/2/2019,41,38.3,37,39,36.1,35,97,92.2,79,15,10.8,8,29.6,29.6,29.4,1
2,1/3/2019,52,40.5,31,38,36.0,30,97,85.4,53,13,7.5,0,29.5,29.4,29.4,1
3,1/4/2019,67,46.0,30,36,33.4,29,96,66.7,31,10,3.8,0,29.6,29.5,29.4,1
4,1/5/2019,73,49.1,32,40,35.1,30,93,66.4,24,13,4.3,0,29.6,29.5,29.5,0


In [5]:
alltrips.head()

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Year,Census Tract Start,Census Tract End
0,6865731e-5e4b-4819-9435-4e62e09a652d,08c8c852-257c-49ad-a14f-727bfc79626d,NaN,NaN,NaN,NaN,NaN,12/13/2018 03:36:12 PM,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,08dc0d9c-735e-4006-b9af-60011509e469,255716b5-4a9f-4efc-9967-7a65b3521bac,bicycle,11491603.0,1658.0,11/30/2018 11:00:00 AM,04/12/2019 12:00:00 PM,12/21/2018 12:41:42 AM,11.0,11.0,5.0,9.0,0.0,2018.0,NaN,NaN
2,aa27d854-9f27-456e-ae49-f268a9b2b533,6b90a827-60a1-47ad-94a4-d33654dbd50d,scooter,170.0,0.0,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000601
3,b7c9f133-966b-46f1-91ce-2682ce7fcf50,1e5234a3-e86b-41e1-a1ad-e98310f3a71c,scooter,90.0,0.0,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453000601
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,caa0325c-7c0b-4909-a573-d83126ecc953,scooter,570.0,826.0,04/29/2019 05:15:00 PM,04/29/2019 05:15:00 PM,04/30/2019 06:45:17 AM,4.0,17.0,1.0,9.0,9.0,2019.0,4.8453e+10,48453001100


In [6]:
# select relevant columns
weather = weather[["Month", "maxtemp", "atemp", "mintemp", "maxhum", "ahum",
                  "minhum", "maxwind", "awind", "minwind","prec"]]
weather = weather.rename(columns={"Month":"date"})

In [7]:
# convert data types where needed
weather['date'] = weather['date'].astype('datetime64')
weather['atemp'] = weather['atemp'].astype('int')
weather['ahum'] = weather['ahum'].astype('int')
weather['awind'] = weather['awind'].astype('int')

In [8]:
# change date into individual columns
weather['day'] = weather['date'].dt.day
weather['month'] = weather['date'].dt.month
weather['day_of_week'] = weather['date'].dt.dayofweek
weather['year'] = weather['date'].dt.year

In [9]:
weather.head()

,date,maxtemp,atemp,mintemp,maxhum,ahum,minhum,maxwind,awind,minwind,prec,day,month,day_of_week,year
0,2019-01-01,46,41,36,97,83,76,21,11,0,0,1,1,1,2019
1,2019-01-02,41,38,37,97,92,79,15,10,8,1,2,1,2,2019
2,2019-01-03,52,40,31,97,85,53,13,7,0,1,3,1,3,2019
3,2019-01-04,67,46,30,96,66,31,10,3,0,1,4,1,4,2019
4,2019-01-05,73,49,32,93,66,24,13,4,0,0,5,1,5,2019


In [10]:
# select relevant columns
alltrips = alltrips[["ID","Vehicle Type", "Trip Duration", 
                     "Trip Distance", "Start Time", "End Time", 
                    "Month", "Hour", "Year"]]

In [11]:
# drop rows with null values
alltrips = alltrips.dropna()

In [12]:
# convert data types where needed
alltrips["Trip Distance"] = alltrips["Trip Distance"].astype("int")
alltrips["Trip Duration"] = alltrips["Trip Duration"].astype("int")
alltrips["month"] = alltrips["Month"].astype("int")
alltrips["hour"] = alltrips["Hour"].astype("int")
alltrips["year"] = alltrips["Year"].astype("int")

In [13]:
# drop skewed data 
alltrips = alltrips[alltrips["Trip Distance"] > 0]
alltrips = alltrips[alltrips["Trip Distance"] <= 27358.8]
alltrips = alltrips[alltrips["Trip Duration"] <= 86400]

In [14]:
alltrips=alltrips.drop(['Month','Hour','Year'],axis=1)
alltrips.head()

,ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,month,hour,year
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,scooter,570,826,04/29/2019 05:15:00 PM,04/29/2019 05:15:00 PM,4,17,2019
5,cd442fb9-058f-46a7-af90-b4adb56fd163,scooter,438,1815,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,4,17,2019
6,e1ac0e42-ab2d-4eac-b019-f085169c1d38,scooter,391,1519,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,4,17,2019
8,1c8b3eab-6aed-4b69-8eca-6b5e0b70a45f,scooter,570,1843,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,4,17,2019
9,f9174176-9d16-431d-af21-ebcab226da31,scooter,668,2829,04/29/2019 05:15:00 PM,04/29/2019 05:30:00 PM,4,17,2019


In [15]:
# split data by vehicle type
scooters = alltrips[alltrips["Vehicle Type"] == "scooter"]
bicycles = alltrips[alltrips["Vehicle Type"] == "bicycle"]

In [16]:
# split start time data to get date information so that we can merge with
# weather data later
new = scooters["Start Time"].str.split("/", n = 2, expand = True) 

In [17]:
new.head()

,0,1,2
4,04,29,2019 05:15:00 PM
5,04,29,2019 05:30:00 PM
6,04,29,2019 05:30:00 PM
8,04,29,2019 05:30:00 PM
9,04,29,2019 05:15:00 PM


In [18]:
# convert data to dates 
scooters['day'] = new[1].astype('int')

C:\Users\Ashley\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
scooters.head()

,ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,month,hour,year,day
4,1d4f5d89-c044-46cb-8674-80d37fa0371f,scooter,570,826,04/29/2019 05:15:00 PM,04/29/2019 05:15:00 PM,4,17,2019,29
5,cd442fb9-058f-46a7-af90-b4adb56fd163,scooter,438,1815,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,4,17,2019,29
6,e1ac0e42-ab2d-4eac-b019-f085169c1d38,scooter,391,1519,04/29/2019 05:30:00 PM,04/29/2019 05:30:00 PM,4,17,2019,29
8,1c8b3eab-6aed-4b69-8eca-6b5e0b70a45f,scooter,570,1843,04/29/2019 05:30:00 PM,04/29/2019 05:45:00 PM,4,17,2019,29
9,f9174176-9d16-431d-af21-ebcab226da31,scooter,668,2829,04/29/2019 05:15:00 PM,04/29/2019 05:30:00 PM,4,17,2019,29


In [20]:
# group data and get the count of scooters per hour per day
scooters_grouped = scooters.groupby(['month','hour','year','day'])['ID'].count().reset_index()

In [21]:
scooters_grouped = scooters_grouped.rename(columns={'ID':'count'})

In [22]:
scooters_grouped.head()

,month,hour,year,day,count
0,1,0,2019,1,342
1,1,0,2019,2,79
2,1,0,2019,3,16
3,1,0,2019,4,82
4,1,0,2019,5,188


In [23]:
# merge scooter data with weather 
scooter_summary = pd.merge(scooters_grouped, weather, how='left', on=['year','month', 'day'])

In [24]:
# clean up merged data
scooter_summary = scooter_summary.dropna()
scooter_summary['maxtemp'] = scooter_summary['maxtemp'].astype('int')
scooter_summary['atemp'] = scooter_summary['atemp'].astype('int')
scooter_summary['mintemp'] = scooter_summary['mintemp'].astype('int')
scooter_summary['maxhum'] = scooter_summary['maxhum'].astype('int')
scooter_summary['ahum'] = scooter_summary['ahum'].astype('int')
scooter_summary['minhum'] = scooter_summary['minhum'].astype('int')
scooter_summary['maxwind'] = scooter_summary['maxwind'].astype('int')
scooter_summary['awind'] = scooter_summary['awind'].astype('int')
scooter_summary['minwind'] = scooter_summary['minwind'].astype('int')
scooter_summary['prec'] = scooter_summary['prec'].astype('int')
scooter_summary['day_of_week'] = scooter_summary['day_of_week'].astype('int')

In [25]:
scooter_summary.head()

,month,hour,year,day,count,date,maxtemp,atemp,mintemp,maxhum,ahum,minhum,maxwind,awind,minwind,prec,day_of_week
0,1,0,2019,1,342,2019-01-01,46,41,36,97,83,76,21,11,0,0,1
1,1,0,2019,2,79,2019-01-02,41,38,37,97,92,79,15,10,8,1,2
2,1,0,2019,3,16,2019-01-03,52,40,31,97,85,53,13,7,0,1,3
3,1,0,2019,4,82,2019-01-04,67,46,30,96,66,31,10,3,0,1,4
4,1,0,2019,5,188,2019-01-05,73,49,32,93,66,24,13,4,0,0,5


In [26]:
bicycles.head()

,ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,month,hour,year
829,30b1f786-7385-49fc-b4f6-153d95a2f1e4,bicycle,882,2349,04/29/2019 12:15:00 PM,04/29/2019 12:30:00 PM,4,12,2019
830,730ac628-160c-49db-9183-bd06d8060db4,bicycle,454,2011,04/29/2019 12:45:00 PM,04/29/2019 01:00:00 PM,4,12,2019
831,0f4cbccc-8f3b-4509-ab87-8400690bc347,bicycle,800,1207,04/29/2019 12:45:00 PM,04/29/2019 01:00:00 PM,4,12,2019
839,21fd650a-278f-43bc-b00b-4aee1abc0558,bicycle,389,1979,04/29/2019 12:45:00 PM,04/29/2019 01:00:00 PM,4,12,2019
861,7998fa97-0b79-46d3-ab2f-1de1411f14e3,bicycle,191,675,04/29/2019 11:45:00 AM,04/29/2019 12:00:00 PM,4,11,2019


In [27]:
# split start time data to get date information so that we can merge with
# weather data later
new = bicycles["Start Time"].str.split("/", n = 2, expand = True) 

In [28]:
bicycles['day'] = new[1].astype('int')

C:\Users\Ashley\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
# group bikes by month, hour, day, and year and get the count
bikes_grouped = bicycles.groupby(['month','hour','year','day'])['ID'].count().reset_index()

In [30]:
bikes_grouped = bikes_grouped.rename(columns={'ID':'count'})

In [31]:
bikes_grouped.head()

,month,hour,year,day,count
0,1,0,2019,1,12
1,1,0,2019,2,2
2,1,0,2019,4,9
3,1,0,2019,5,11
4,1,0,2019,6,12


In [32]:
# merge bike data with weather
bikes_summary = pd.merge(bikes_grouped, weather, how='left', on=['year','month', 'day'])

In [33]:
# clean up merged data 
bikes_summary = bikes_summary.dropna()
bikes_summary['maxtemp'] = bikes_summary['maxtemp'].astype('int')
bikes_summary['atemp'] = bikes_summary['atemp'].astype('int')
bikes_summary['mintemp'] = bikes_summary['mintemp'].astype('int')
bikes_summary['maxhum'] = bikes_summary['maxhum'].astype('int')
bikes_summary['ahum'] = bikes_summary['ahum'].astype('int')
bikes_summary['minhum'] = bikes_summary['minhum'].astype('int')
bikes_summary['maxwind'] = bikes_summary['maxwind'].astype('int')
bikes_summary['awind'] = bikes_summary['awind'].astype('int')
bikes_summary['minwind'] = bikes_summary['minwind'].astype('int')
bikes_summary['prec'] = bikes_summary['prec'].astype('int')
bikes_summary['day_of_week'] = bikes_summary['day_of_week'].astype('int')

In [34]:
bikes_summary.head()

,month,hour,year,day,count,date,maxtemp,atemp,mintemp,maxhum,ahum,minhum,maxwind,awind,minwind,prec,day_of_week
0,1,0,2019,1,12,2019-01-01,46,41,36,97,83,76,21,11,0,0,1
1,1,0,2019,2,2,2019-01-02,41,38,37,97,92,79,15,10,8,1,2
2,1,0,2019,4,9,2019-01-04,67,46,30,96,66,31,10,3,0,1,4
3,1,0,2019,5,11,2019-01-05,73,49,32,93,66,24,13,4,0,0,5
4,1,0,2019,6,12,2019-01-06,75,53,34,100,84,53,17,7,0,0,6


In [35]:
# add numeric values for vehicle types
scooter_summary['vehicle_type'] = 'scooter'
scooter_summary['vehicle_encoded'] = 0
bikes_summary['vehicle_type'] = 'bicycle'
bikes_summary['vehicle_encoded'] = 1

In [36]:
# merge both vehicle dataframes
all_summary = pd.concat([scooter_summary,bikes_summary])

In [37]:
all_summary.head()

,month,hour,year,day,count,date,maxtemp,atemp,mintemp,maxhum,ahum,minhum,maxwind,awind,minwind,prec,day_of_week,vehicle_type,vehicle_encoded
0,1,0,2019,1,342,2019-01-01,46,41,36,97,83,76,21,11,0,0,1,scooter,0
1,1,0,2019,2,79,2019-01-02,41,38,37,97,92,79,15,10,8,1,2,scooter,0
2,1,0,2019,3,16,2019-01-03,52,40,31,97,85,53,13,7,0,1,3,scooter,0
3,1,0,2019,4,82,2019-01-04,67,46,30,96,66,31,10,3,0,1,4,scooter,0
4,1,0,2019,5,188,2019-01-05,73,49,32,93,66,24,13,4,0,0,5,scooter,0


In [38]:
# create new csvs of cleaned/transformed data 
all_summary.to_csv("all_summary.csv")

In [39]:
scooter_summary.to_csv("scooter_summary.csv")

In [40]:
bikes_summary.to_csv("bikes_summary.csv")

In [41]:
alltrips.to_csv("alltrips_cleaned.csv")

In [42]:
# connect to local postgres
rds_connection_string = "postgres:<password>@localhost:5432/micromobility"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
# check if table exists in animals database
engine.table_names()

['weather']

In [12]:
# load dataframe to postgres database
weather.to_sql(name = 'weather', con = engine, if_exists="append", index=False)